# chapter 5. 분류1 로지스틱 회귀
1. 분류란? 데이터를 보고 어느 그룹에 속해있는지 라벨을 붙이는 작업
2. sigmoid함수: 모델 점수(z)를 확률 0~1로 바꾸는 변환기
3. 로지스틱: sigmoid함수를 이용한 확률 기반 분류기
4. 임계값: 임계값에 따라 Precision, Recall 줄다리기

| 개념 | 핵심 | 코드 |
|---|---|---|
| 로지스틱 회귀 | 선형 점수 z를 확률 p로 바꿔서 분류한다 | sigmoid 시각화 |
| 전처리 + 누수 방지 | train으로만 fit, Pipeline/ColumnTransformer로 묶는다 | ColumnTransformer + Pipeline |
| predict vs predict_proba | predict는 0/1, predict_proba는 확률 | predict_proba[:,1] |
| 임계값(threshold) | 확률 기준을 바꾸면 FP/FN이 바뀐다 | y_hat = (proba>=thr) |
| Precision/Recall | 오탐/누락 관점의 줄다리기 | confusion_matrix, precision_score, recall_score |
| 확률 분포 해석 | 모델이 좋아질수록 0/1 확률 분포가 분리된다 | 클래스별 히스토그램 |
| AUC/Accuracy 비교 | AUC는 threshold에 덜 민감한 확률 품질 지표다 | roc_auc_score, accuracy_score |

# part 0. 분류를 다루는 이유
분류는 데이터를 보고 어떠한 그룹에 속해하는지 라벨을 붙이는 작업이다.


대표적인 예시
| 알고리즘 | 핵심 아이디어 | 기준 |
|---------|-------------|--------|
| **로지스틱 회귀** | 점수 → 확률 → 분류 | 확률 |
| **KNN** | 가까운 이웃끼리 다수결 | 거리 |
| **결정 트리** | 질문을 반복해서 분류 | 규칙 |

# part 1. 로지스틱 회귀란?
확률을 먼저 만들고 기준값&임계값으로 판단한다.

1. 모델은 먼저 점수 z를 만든다: z = w·x + b  
2. sigmoid가 점수를 확률로 바꾼다: p = 1 / (1 + exp(-z))  
3. 임계값(threshold)으로 0/1을 결정한다: p >= 0.5 이면 1  
4. 학습은 log loss(교차엔트로피)를 줄이는 방향으로 w,b를 찾는다.


## 개념설명
- 선형회귀처럼 보이지만, 마지막에 sigmoid를 붙여서 출력이 0~1 확률이 된다.
- Decision boundary(경계선)는 threshold를 어디에 두느냐에 따라 달라진다.
- 그래서 분류는 모델 학습뿐 아니라 임계값 선택까지가 한 세트다.


## sigmoid함수 - 점수를 확률로 바꾸는 변환기
모델이 내부적으로 계산한 점수(z)는 **-∞ ~ +∞** 범위이다.\
하지만 우리가 원하는건 한정된 범위의 %\
이런 상황에서 sigmoid함수가 0~1사이의 학률로 바꾼다.

$\sigma(z) = \frac{1}{1 + e^{-z}}$


해석법: 
| z 값 | sigmoid(z) | 해석 |
|------|-----------|------|
| +10 | ≈ 1.00 | 양성일 확률이 높다\ |
| 0 | 0.50 | 반반 <br>가장 애매한 지점 |
| -10 | ≈ 0.00 | 음성일 확률이 높다 |

> 로지스틱 회귀가 가장 중요한것은 **기준값(임계값)**을 잘 정하는게 중요하다

# part2. 타이타닉 데이터 전처리 해보기
- 데이터: 실제 타이타닉호 생존자 데이터
- 목표: 승객의 나이, 성별, 선실 등급 등을 보고 **생존 여부(0=사망, 1=생존)** 예측
- 주요 컬럼:
  - Pclass: 선실 등급 (1=일등석, 2=이등석, 3=삼등석)
  - Sex: 성별
  - Age: 나이
  - Fare: 운임 요금
- **Target(정답)**: `Survived` 생존자(0: 사망/1: 생존)

## 데이터 전처리 과정
- 데이터 로드&카피
- 결측치 처리 및 기타 정리
- 데이터 크기 및 비율 확인
- traning/test 나눔 (전처리는 fit 훈련 데이터만 해야 하기 때문)
- 숫자형과 범주형을 나누기
- 파이프라인을 통해 인코딩, 스케일링 실행
- 이후 파이프라인으로 나눈거 함칩

## 전처리 핵심요약
- 전처리는 모델 학습에 꼭 필요하지만, test 정보를 미리 보면 데이터 누수다.
- 해결: 전처리 + 모델을 Pipeline으로 묶고, train 데이터로만 fit 되게 만든다.
- 숫자형: 결측치 대체(median) + 스케일링(StandardScaler)
- 범주형: 결측치 대체(most_frequent) + 원핫인코딩(OneHotEncoder)

In [14]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
titanic_df = pd.read_csv(url)
print("데이터 크기")
print(titanic_df.shape)
print("\n데이터 컬럼명")
print(titanic_df.columns)


titanic_df.head()

데이터 크기
(891, 12)

데이터 컬럼명
Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='str')


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [15]:
# 타입/결측 요약
titanic_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    str    
 4   Sex          891 non-null    str    
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    str    
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    str    
 11  Embarked     889 non-null    str    
dtypes: float64(2), int64(5), str(5)
memory usage: 83.7 KB


In [16]:
# 컬럼별 결측 개수
titanic_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [17]:
# 숫자 요약
titanic_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [18]:
# 범주형 요약
titanic_df.describe(include="object")

C:\Users\joon5\AppData\Local\Temp\ipykernel_52048\2669026564.py:2: Pandas4Warning:

For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.



,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Braund, Mr. Owen Harris",male,347082,G6,S
freq,1,577,7,4,644


In [19]:
# 범주 컬럼 값 확인
display(titanic_df["Sex"].value_counts(dropna=False))
display(titanic_df["Embarked"].value_counts(dropna=False))
titanic_df["Cabin"].isna().mean()  # Cabin 결측 비율

Sex
male      577
female    314
Name: count, dtype: int64

Embarked
S      644
C      168
Q       77
NaN      2
Name: count, dtype: int64

np.float64(0.7710437710437711)

In [20]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression

# 데이터 로드
titanic_df = pd.read_csv(url)


# =======================================
# 2) 타깃(y)과 입력(X) 분리
# y: 생존 여부(0/1) : 타겟값
y_titanic_df = titanic_df["Survived"]

# X: y를 제외한 나머지 피처들
# 모델에 넣을 입력 데이터(특성들)
X_titanic_df = titanic_df.drop("Survived", axis=1).copy()
# =======================================

# =======================================
# 3) Cabin 파생변수 만들기: Cabin1
# Cabin은 결측이 많고 값이 복잡해 그대로 쓰기 어렵다.
# 그래서 첫 글자만 뽑아 범주형으로 만든다.
# 결측치는 'N'으로 채워 "Cabin 정보 없음"을 하나의 범주로 만든다.
X_titanic_df["Cabin1"] =(
    X_titanic_df["Cabin"]
    .fillna("N")            # 결측치를 'N'으로 채움
    .astype(str)            # 문자열 변환
    .str[:1]                # 첫 글자만 추출 (예: 'C85' -> 'C')
)
# =======================================

# =======================================
# 4) 모델에 쓰기 애매한 컬럼 제거
# PassengerId: 단순 식별자(정보 없음)
# Name/Ticket: 문자열이 너무 복잡(처리 없이 넣으면 어려움)
# Cabin: 원본은 결측치 많고 형태도 복잡 -> Cabin1로 대체
X_titanic_df = (
    X_titanic_df
    .drop(
        [
            "PassengerId", 
            "Name", 
            "Ticket", 
            "Cabin"
            ], axis=1
            ))
# =======================================

# =======================================
# 5) 데이터 간단 확인
print(f"데이터 크기: {X_titanic_df.shape}")
# 클래스 비율은 항상 확인하는 습관을 들이자
print(f"생존 비율:\n{y_titanic_df.value_counts(normalize=True)}")
display(X_titanic_df.head())
# =======================================

# =======================================
# 6) train/test 분리
X_train, X_test, y_train, y_test = train_test_split(
    X_titanic_df,           # 입력값
    y_titanic_df,           # 타겟값
    test_size=0.2,          # 20%를 테스트로
    stratify=y_titanic_df,  # 클래스 비율을 train/test에 비슷하게 유지
    random_state=42         # 재현성(매번 같은 split)
)
# =======================================

# =======================================
# 7) 숫자/범주 컬럼 구분
numeric_features = ([
    "Pclass", 
    "Age", 
    "SibSp", 
    "Parch", 
    "Fare"
])
categorical_features = ([
    "Sex", 
    "Embarked", 
    "Cabin1"
])
# =======================================

# =======================================
# 8) 숫자형 전처리 파이프라인
# 결측치는 중앙값으로 채움(이상치에 평균보다 덜 흔들림)
# 스케일 표준화(StandardScaler): 평균 0, 표준편차 1로 맞춤
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])
# =======================================

# =======================================
# 9) 범주형 전처리 파이프라인
# 결측치는 최빈값으로 채움
# OneHotEncoder: 범주를 0/1 더미 변수로 바꿈
# handle_unknown="ignore": 테스트에서 처음 보는 범주가 나와도 에러 없이 처리
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])
# =======================================

# =======================================
# 10) 컬럼별 전처리 결합
# num 컬럼엔 numeric_transformer
# cat 컬럼엔 categorical_transformer
# remainder="drop": 지정하지 않은 컬럼은 버림
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ],
    remainder="drop"
)
# =======================================

# =======================================
# 11) 최종 파이프라인: 전처리 + 모델
# max_iter=5000: 수렴이 느릴 때 반복 횟수 늘려 경고를 줄임
lr = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", LogisticRegression(max_iter=5000, random_state=42)),
])
# =======================================

데이터 크기: (891, 8)
생존 비율:
Survived
0    0.616162
1    0.383838
Name: proportion, dtype: float64


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Cabin1
0,3,male,22.0,1,0,7.2500,S,N
1,1,female,38.0,1,0,71.2833,C,C
2,3,female,26.0,0,0,7.9250,S,N
3,1,female,35.0,1,0,53.1000,S,C
4,3,male,35.0,0,0,8.0500,S,N


## 지금까지 배운 모델링을 위해 필요한 전처리 정리
1) 인코딩(Encoding) 문자(범주)를 모델이 계산할 수 있는 숫자 형태로 바꾸는 작업\
예시: OneHotEncoder(handle_unknown="ignore")

2) 스케일링(Scaling): 숫자 피처들의 단위/크기 차이를 맞춰서, 모델 학습이 안정적으로 되게 만든다.\
예시: StandardScaler()

3) 정규화(Normalization): 값의 범위를 0~1처럼 일정 범위로 맞추거나, 벡터 길이를 1로 맞추는 전처리

4) 데이터 누수(Data Leakage)와 파이프라인 : 테스트 정보가 학습 과정에 섞이면 성능이 **좋아 보이는 착시**가 생긴다. 파이프라인은 이를 막는 대표 도구다.\
예시: Pipeline([...])

5) ColumnTransformer(...): 숫자/범주 컬럼에 서로 다른 전처리를 "같은 기준으로" 적용한다.\
컬럼 그룹별 전처리 결합

## Part 3. 학습(fit)과 확률 예측(predict_proba)
- fit: train으로 규칙 학습
- predict_proba: 클래스별 확률 출력(분류에서 매우 중요)
- predict: 내부적으로 threshold=0.5로 0/1을 만든 결과

## 개념설명
- 분류 모델은 "확률을 먼저 만들고" 마지막에 0/1로 자른다.
- 그래서 성능을 개선하려면 확률을 어떻게 쓸지(임계값/비용/목표)를 같이 생각해야 한다.

```
fit(X_train, y_train) # (X_train, y_train)로 로지스틱 회귀 모델의 가중치(규칙)를 학습한다
y_proba = lr.predict_proba(X_test) # X_test 각 샘플에 대해 클래스별 예측확률([0확률, 1확률])을 반환한다
```


In [21]:
lr.fit(X_train, y_train)
y_proba = lr.predict_proba(X_test)[:, 1]
pd.DataFrame({"예측 확률 (생존)": y_proba[:10].round(4)})


,예측 확률 (생존)
0,0.0669
1,0.0467
2,0.1610
3,0.0345
4,0.6714
5,0.4439
6,0.7027
7,0.2944
8,0.3258
9,0.1268


## 코드 해설 4: 기본 예측(predict)으로 Accuracy 확인
threshold=0.5 기본 규칙으로 0/1 예측을 만들고 정확도를 본다.
```
y_pred = lr.predict(X_test) # 내부적으로 임계값 0.5 기준으로 확률을 0/1 레이블로 변환해 예측한다
accuracy_score(y_test, y_pred) # 정답(y_test)과 예측(y_pred)이 일치한 비율(정확도)을 계산한다
```
기본 threshold=0.5에서 Accuracy를 확인해 베이스라인을 잡았다

In [22]:
from sklearn.metrics import accuracy_score

# 기본 예측 (임곗값 0.5)
y_pred = lr.predict(X_test)
print(f"기본 Accuracy (threshold=0.5): {accuracy_score(y_test, y_pred):.3f}")

기본 Accuracy (threshold=0.5): 0.816


### Decision Boundary(경계선) 의미
확률이 0.5가 되는 지점이 경계선을 의미한다.\
**경계선에서 멀어질수록** 확률이 0 또는 1에 가까워져 **모델이 더 확신**하게 되지만\
반대로 **경계선에서 가까워질수록** 확률이 0.5 근처라 **모델이 헷갈리는** 영역이다.

즉, 0.5 근처 값이 많으면 **모델이 확신 못 하는 샘플이 많다**는 신호일 수 있다.\
다만 "애매한 값이 많다 = 무조건 나쁜 모델"이 의미하지 않고\
제대로 된 평가는 평가지표를 확인해야 한다.

## Part 4. 임계값(Threshold)에 따른 Precision/Recall 줄다리기
임계값(threshold)은 "양성(1)이라고 판정할 기준"이다.\
이를 올리거나 내리면 FP/FN이 바뀌면서 Precision과 Recall이 서로 줄다리기한다.

- threshold를 낮추면 양성으로 더 많이 잡는다 → Recall↑, Precision↓ 경향
- threshold를 높이면 양성으로 덜 잡는다 → Precision↑, Recall↓ 경향
- 혼동행렬(confusion matrix)에서 FP/FN이 어떻게 바뀌는지로 이해한다

### predict vs predict_proba
predict_proba(X): 각 클래스 확률을 준다: [P(0), P(1)]

predict(X): 내부적으로 predict_proba()의 P(1)을 만든 다음 기본 임계값 0.5로 잘라서 0/1을 반환한다.

| 메서드 | 반환값 | 예시 |
|--------|--------|------|
| `predict()` | 0 또는 1 | [0, 1, 0, 1] |
| `predict_proba()` | 각 클래스의 확률 | [[0.92, 0.08], [0.27, 0.73]] |

## 코드 해설 5: threshold 0.3/0.5/0.7 비교 + 혼동행렬 확인
임계값을 바꾸면서 Precision/Recall과 confusion matrix가 어떻게 달라지는지 직접 확인한다

사용 함수/메서드 요약:
```
precision_score(y_test, y_pred) # 예측을 1이라고 한 것 중에서 실제로 1인 비율(정밀도)을 계산한다
recall_score(y_test, y_pred) # 실제 1인 것 중에서 모델이 1로 맞춘 비율(재현율)을 계산한다
confusion_matrix(y_test, y_pred) # TN/FP/FN/TP를 표 형태로 집계해 어떤 오류가 많은지 보여준다

```

threshold를 {thr}로 조정했더니 FP/FN이 바뀌어 Precision/Recall이 함께 변화했으므로, 목적(오탐 최소/누락 최소)에 맞춰 threshold를 선택한다


In [23]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix
import pandas as pd

for thr in [0.3, 0.5, 0.7]:
    y_hat = (y_proba >= thr).astype(int)

    print(f"Threshold={thr}")
    print(f"  Precision: {precision_score(y_test, y_hat, zero_division=0):.4f}")
    print(f"  Recall   : {recall_score(y_test, y_hat, zero_division=0):.4f}")

    cm = confusion_matrix(y_test, y_hat)
    cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Pred 0", "Pred 1"])
    cm_df
    print()

Threshold=0.3
  Precision: 0.6506
  Recall   : 0.7826

Threshold=0.5
  Precision: 0.8103
  Recall   : 0.6812

Threshold=0.7
  Precision: 0.8974
  Recall   : 0.5072



- 임계값을 낮추면 (예: 0.3)\
더 쉽게 1이라고 판정 → 1이 많아진다.\
→ 30%만 되어도 생존했다 FN 감소, FP 증가\
→ Recall 증가, Precision 감소

- 임계값을 높이면 (예: 0.7)\
더 엄격하게 1 판정 → 1이 적어진다.\
→ 30%만 되어도 생존했다 FN 증가, FP 감소\
→ Recall 감소, Precision 증가

> FN이 증감, FP가 증감함에 따라 recall이 증가하나? Precision이 증가하나? 를 딱딱 알아야함

예시\
암진단에서 중요한것은? :FN(누락)을 잡는게 더 중요하다\
스팸에서 중요한것은?: FP(오탐)을 잡는게 더 중요하다.

> 즉, 임계값을 어떻게 잡는가에 대한것은 도메인과 상황을 기준으로 잡아야 한다.

## 코드 해설 6: 여러 threshold에서 Precision/Recall/F1을 표로 만들기
threshold를 0.1~0.9로 바꿔가며 지표가 어떻게 움직이는지 한 번에 본다

```
f1_score(y_test, y_pred) # precision과 recall의 조화평균으로 두 지표 균형 성능을 계산한다

```

threshold를 여러 값으로 스윕해보니 F1이 가장 높은 구간이 있었고, 그 근처를 운영 임계값 후보로 잡을 수 있다

In [24]:
from sklearn.metrics import f1_score
import plotly.graph_objects as go

thr_list = np.round(np.arange(0.1, 1.0, 0.1), 2)
rows = []
for thr in thr_list:
    y_hat = (y_proba >= thr).astype(int)
    rows.append([
        thr,
        precision_score(y_test, y_hat, zero_division=0),
        recall_score(y_test, y_hat, zero_division=0),
        f1_score(y_test, y_hat, zero_division=0)
    ])

thr_df = pd.DataFrame(rows, columns=["Threshold", "Precision", "Recall", "F1"])
thr_df

,Threshold,Precision,Recall,F1
0,0.1,0.456522,0.913043,0.608696
1,0.2,0.600000,0.826087,0.695122
2,0.3,0.650602,0.782609,0.710526
3,0.4,0.698630,0.739130,0.718310
4,0.5,0.810345,0.681159,0.740157
5,0.6,0.851064,0.579710,0.689655
6,0.7,0.897436,0.507246,0.648148
7,0.8,0.954545,0.304348,0.461538
8,0.9,0.916667,0.159420,0.271605


# part 5. 확률 분포로 모델을 해석하기
좋은 모델일수록 \
y=0(사망) 확률은 0 근처에 몰리고\
y=1(생존) 확률은 1 근처에 몰린다.

두 분포가 많이 겹치면 "애매한 구간"이 많다는 것을 의미한다.\
이때 피처를 더 추가하면 분포가 더 분리될 수 있다. (예: Sex 추가)

분류 성능은 "0/1 맞춘 개수" 뿐만 아닌, 확률이 얼마나 깔끔하게 분리되는지도 중요하다.\
만약, 확률 품질 자체를 보고 싶을 때 AUC를 같이 봐야한다.

## 코드 해설 8: 피처 조합별 확률 분포(히스토그램) 비교
피처를 늘릴수록 생존/사망 확률 분포가 더 잘 분리되는지 확인한다

사용 함수/메서드 요약:
```
make_lr_pipeline(파이프라인 생성), 
predict_proba,
roc_auc_score, 
plotly Histogram
```


In [25]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import roc_auc_score

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression

all_numeric = {"Pclass", "Age", "SibSp", "Parch", "Fare"}
all_categorical = {"Sex", "Embarked", "Cabin1"}

def make_lr_pipeline(selected_features):
    selected_features = list(selected_features)
    num_feats = [f for f in selected_features if f in all_numeric]
    cat_feats = [f for f in selected_features if f in all_categorical]

    numeric_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ])

    categorical_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, num_feats),
            ("cat", categorical_transformer, cat_feats),
        ],
        remainder="drop",
    )

    return Pipeline(steps=[
        ("preprocess", preprocessor),
        ("model", LogisticRegression(max_iter=5000, random_state=42)),
    ])


configs = [
    ("Pclass만", ["Pclass"]),
    ("Pclass + Sex", ["Pclass", "Sex"]),
    ("전체 피처(전처리 한거임)", X_train.columns.tolist()),
]

titles, probas = [], []
for name, feats in configs:
    m = make_lr_pipeline(feats)
    m.fit(X_train, y_train)
    p = m.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, p)
    titles.append(f"{name} (AUC={auc:.3f})")
    probas.append(p)

fig = make_subplots(rows=3, cols=1, subplot_titles=titles)

for i, p in enumerate(probas, 1):
    fig.add_trace(go.Histogram(
        x=p[y_test == 0], name="사망 (0)",
        opacity=0.6, marker_color="cyan",
        xbins=dict(start=0, end=1, size=0.05),
        legendgroup="died", showlegend=(i == 1)
    ), row=i, col=1)

    fig.add_trace(go.Histogram(
        x=p[y_test == 1], name="생존 (1)",
        opacity=0.6, marker_color="magenta",
        xbins=dict(start=0, end=1, size=0.05),
        legendgroup="survived", showlegend=(i == 1)
    ), row=i, col=1)

fig.update_layout(
    height=900,
    barmode="overlay", template="plotly_dark",
    title="모델이 좋아질수록 확률 분포가 분리!",
)

for i in range(1, 4):
    fig.update_xaxes(range=[0, 1], row=i, col=1)

fig.show()

# part 6. AUC와 Accuracy를 같이 보는 이유
- Accuracy: threshold=0.5로 잘랐을 때의 맞춘 비율
- AUC: threshold를 바꿔도 전반적으로 "양성을 더 높은 확률로 주는가"를 보는 지표

확률을 조정한다면 AUC가 특히 유용하다.

## 코드 해설 9: 피처별 모델 성능(AUC/Accuracy) 비교표
피처를 다르게 썼을 때 AUC와 Accuracy가 어떻게 달라지는지 표로 정리한다

사용 함수/메서드 요약:
```
roc_auc_score(y_test, y_proba) # 임계값을 여러 개로 바꿔가며 ‘양성 점수를 더 높게 주는 능력’을 AUC로 요약한다
accuracy_score(y_test, y_pred) # 정답(y_test)과 예측(y_pred)이 일치한 비율(정확도)을 계산한다
```


In [26]:
from sklearn.metrics import roc_auc_score, accuracy_score
import pandas as pd

configs_ext = [
    ("Pclass만", ["Pclass"]),
    ("Sex만", ["Sex"]),
    ("Pclass + Sex", ["Pclass", "Sex"]),
    ("전체 피처", X_train.columns.tolist()),
]

rows = []
for name, feats in configs_ext:
    m = make_lr_pipeline(feats)
    m.fit(X_train, y_train)
    p = m.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, p)
    acc = accuracy_score(y_test, (p >= 0.5).astype(int))
    rows.append([name, len(feats), f"{auc:.3f}", f"{acc:.3f}"])

pd.DataFrame(rows, columns=["모델", "피처 수", "AUC", "Accuracy"])

,모델,피처 수,AUC,Accuracy
0,Pclass만,1,0.668,0.642
1,Sex만,1,0.753,0.777
2,Pclass + Sex,2,0.822,0.777
3,전체 피처,8,0.841,0.816


# 오늘의 배운 내용 정리

1. 로지스틱 회귀는 선형 점수 z를 sigmoid로 확률 p로 바꿔 분류한다.
2. predict_proba는 확률을 주고, predict는 threshold=0.5로 자른 결과다.
3. 전처리는 train으로만 fit 해야 하며 Pipeline/ColumnTransformer가 누수를 막는 기본 도구다.
4. threshold를 낮추면 Recall이 오르고 Precision이 내려가는 경향이 있다(반대도 성립).
5. 혼동행렬에서 FP/FN이 어떻게 바뀌는지로 Precision/Recall 변화를 이해한다.
6. F1은 Precision과 Recall의 균형을 보는 요약 지표다.
7. 확률 분포가 더 분리될수록 모델이 좋아졌다고 해석하기 쉽다.
8. Accuracy는 특정 threshold 결과이고, AUC는 확률 품질을 더 넓게 본다.
9. 분류는 "모델 학습 + 임계값 선택"까지가 한 세트다.